Using GMAPS to Plot Visual Comparison of Library Visitors versus COVID Cases Reported for 2020-2021

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time
import json

# Import API key
from bz_api_keys import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

In [2]:
# Open the merged Library Visitors csv file
library_path = "../Data/merged_library_covid_outer.csv"

# Read the  data and the study results
df = pd.read_csv(library_path)
df = df.drop(['Unnamed: 0'], axis = 1)
df.head()

,ADDRESS,CITY,ZIP,JANUARY,FEBRUARY,MARCH,APRIL,MAY,JUNE,JULY,AUGUST,SEPTEMBER,OCTOBER,NOVEMBER,DECEMBER,YTD,TYPE,YEAR,LAT,LNG
0,3401 W. Foster Ave.,Chicago,60625,8214.0,7614.0,8460.0,7414.0,7044.0,7970.0,9664.0,9069.0,7053.0,8012.0,6571.0,5924.0,93009.0,Circulation,2019,41.971155,-87.701816
1,13281 S. Corliss Ave.,Chicago,60827,378.0,326.0,332.0,270.0,320.0,325.0,354.0,229.0,234.0,377.0,369.0,500.0,4014.0,Circulation,2019,41.650765,-87.633087
2,5055 S. Archer Ave.,Chicago,60632,5365.0,5019.0,5417.0,5349.0,4833.0,4627.0,5356.0,4793.0,4537.0,5263.0,4051.0,3415.0,58025.0,Circulation,2019,41.810038,-87.711251
3,5615 W. Race Ave.,Chicago,60644,215.0,590.0,706.0,736.0,806.0,983.0,1134.0,1236.0,1193.0,1361.0,1286.0,1082.0,11328.0,Circulation,2019,41.881113,-87.756863
4,6100 W. Irving Park Rd.,Chicago,60634,9678.0,9764.0,10845.0,10541.0,9763.0,11442.0,11634.0,11063.0,9379.0,9932.0,8567.0,6492.0,119100.0,Circulation,2019,41.944967,-87.797373


In [3]:
# Sum up 2020 and 2021 COVID Cases
df = df.sort_values(by = ['ZIP'], axis = 0)
cases = df[df["TYPE"] == "Cases"]
cases_2020 = cases[cases["YEAR"] == 2020]
cases_2021 = cases[cases["YEAR"] == 2021]
cases_sum = cases_2020.rename({"YTD":"YTD_2020"}, axis = 1)
cases_sum["YTD_2021"] = np.array(cases_2021["YTD"])
ytd_sum = np.array(cases_2020["YTD"]) + np.array(cases_2021["YTD"])
cases_sum["YTD_SUM_FINAL"] = ytd_sum

# Sum up 2020 and 2021 Visitors to the library
df = df.sort_values(by = ['ZIP'], axis = 0)
visitors = df[df["TYPE"] == "Visitors"]
visitors = visitors[visitors["ZIP"] != 60611]
visitors_2020 = visitors[visitors["YEAR"] == 2020]
visitors_2021 = visitors[visitors["YEAR"] == 2021]
visitors_sum = visitors_2020.rename({"YTD":"YTD_2020"}, axis = 1)
visitors_sum["YTD_2021"] = np.array(visitors_2021["YTD"])
ytd_sum = np.array(visitors_2020["YTD"]) + np.array(visitors_2021["YTD"])
visitors_sum["YTD_SUM"] = ytd_sum

# Sum same zip codes
zipcode_sum = pd.DataFrame(visitors_sum["ZIP"].unique())
zipcode_sum["YTD_SUM_FINAL"] = np.repeat(0, len(zipcode_sum))
zipcode_sum = zipcode_sum.set_index(0)
for zipcode in visitors_sum["ZIP"].unique():
    visitors_sum_zip = visitors_sum[visitors_sum["ZIP"] == zipcode]
    ytd_sum = visitors_sum_zip["YTD_SUM"].sum()
    zipcode_sum.loc[zipcode] = ytd_sum
zipcode_sum

# Merge
zipcode_sum["ZIP"] = zipcode_sum.index
visitors_sum = pd.merge(visitors_sum, zipcode_sum, how = "left", on = "ZIP")
visitors_sum = visitors_sum.drop_duplicates(subset=['ZIP'])

# Subset for shared zip codes
shared_zips =  list(set(cases_sum['ZIP']) & set(visitors_sum['ZIP']))
visitors_sum = visitors_sum[visitors_sum['ZIP'].isin(shared_zips)]
cases_sum = cases_sum[cases_sum['ZIP'].isin(shared_zips)]

In [4]:
# Store latitude and longitude in locations
# Plotting two differnt heatmaps so the points being compared are cetered around the priomary coorididates
clean_lib_visitors = visitors_sum
clean_lib_cases = cases_sum
clean_lib_visitors[["LNG1"]] = clean_lib_visitors[["LNG"]] - 0.005
clean_lib_visitors[["LNG2"]] = clean_lib_visitors[["LNG"]] + 0.005

# visitors_locations
locations1 = clean_lib_visitors[["LAT", "LNG1"]]
locations2 = clean_lib_visitors[["LAT", "LNG2"]]

# Convert to float
visitors_map_weight = clean_lib_visitors["YTD_SUM_FINAL"].astype(float)
cases_map_weight = clean_lib_cases["YTD_SUM_FINAL"].astype(float)
#print(visitors_map_weight)
#print(cases_map_weight)

Library Visitors and COVID Cases Heatmap
* Use the Lat and Lng as locations and Library Vistiors/COVID Cases as the weight.
* Add Heatmap layer to map.

In [5]:
# Plot Heatmap
fig = gmaps.figure()

# Create heatmap layer 1 for the number of Library Visitors
heat_layer = gmaps.heatmap_layer(locations1, weights=(visitors_map_weight/5000), 
                                 dissipating=False, max_intensity=10,
                                 point_radius=0.005)

# Add layer
fig.add_layer(heat_layer)

# Create heatmap layer 2 for the number of COVID Cases
heat_layer2 = gmaps.heatmap_layer(locations2, weights=(cases_map_weight/600), 
                                 dissipating=False, max_intensity=10,
                                 point_radius=0.005, gradient = [
    "rgba(0, 255, 255, 0)",
    "rgba(0, 255, 255, 1)",
    "rgba(0, 191, 255, 1)",
    "rgba(0, 127, 255, 1)",
    "rgba(0, 63, 255, 1)",
    "rgba(0, 0, 255, 1)",
    "rgba(0, 0, 223, 1)",
    "rgba(0, 0, 191, 1)",
    "rgba(0, 0, 159, 1)",
    "rgba(0, 0, 127, 1)",
    "rgba(63, 0, 91, 1)",
    "rgba(127, 0, 63, 1)",
    "rgba(191, 0, 31, 1)",
    "rgba(255, 0, 0, 1)",
  ])

# Add layer
fig.add_layer(heat_layer2)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))